In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_json('/kaggle/input/wildfiredata/wildfire_collection_feature_engineering_final.json')

In [ ]:
df.info()

In [ ]:
df.drop(columns='_id', inplace=True)

In [ ]:
from datetime import datetime

def return_date(date_string):
    #date_dict = eval(date_string)  # Convert the string to a dictionary
    date_value = date_string['$date']  # Get the value of the '$date' key
    return datetime.strptime(date_value, "%Y-%m-%dT%H:%M:%S.%fZ").date()  # Convert the string to a date object

df['rep_date'] = df['rep_date'].apply(return_date)
df.head()

In [ ]:
df['cfb'] = df['cfb'].astype(float)

Split train and test data

In [ ]:
import datetime

split_date = datetime.date(2023, 1, 1)
train_data = df[df['rep_date'] < split_date]
test_data = df[df['rep_date'] >= split_date]

In [ ]:
train = train_data.reset_index()
train = (train.merge((train[['rep_date']].drop_duplicates(ignore_index=True).rename_axis('time_idx'))\
                     .reset_index(), on = ['rep_date'])).drop("rep_date", axis=1)

In [ ]:
# Check for duplicates
duplicates = train[train.duplicated(['locality', 'time_idx'], keep=False)]
print("Duplicate rows:")
print(duplicates)

In [ ]:
# Aggregate duplicate rows (using mean as an example)
train_unique = train.groupby(['locality', 'time_idx']).mean().reset_index()

# Verify that we now have unique combinations
assert train_unique.duplicated(['locality', 'time_idx']).sum() == 0, "Still have duplicates after aggregation"

In [ ]:
# Create a complete time series for each locality
full_index = pd.MultiIndex.from_product(
    [train_unique['locality'].unique(), 
     range(train_unique['time_idx'].min(), train_unique['time_idx'].max() + 1)],
    names=['locality', 'time_idx']
)

# Reindex the dataframe and reset index to reintroduce locality and time_idx as columns
train_filled = train_unique.set_index(['locality', 'time_idx']).reindex(full_index).reset_index()

# Check the presence of 'locality' column
print(train_filled.columns)

# Forward fill the missing values within each group, but keep locality and time_idx intact
train_filled.update(train_filled.groupby('locality').ffill())

# If needed, backward fill remaining NaN values after forward fill
train_filled.update(train_filled.groupby('locality').bfill())

# Resulting DataFrame
print(train_filled)

In [ ]:
train_filled.info()

### Transform Dataset to TimeSeriesDataset of Pytorch

In [ ]:
train_filled['time_idx'].nunique()

In [ ]:
all_features = ['cfb', 'locality', 'temp', 'wd',
       'elev', 'rh', 'pcuring', 'day', 'ros', 'year', 'month', 'hfi', 'tfc0',
       'sfl', 'bui_lag2', 'bui', 'cfl', 'sfc0', 'dmc_lag5', 'dmc', 'sfc',    
       'dmc_lag7', 'bui_lag4', 'bfc', 'tfc', 'isi', 'bui_lag5', 'fwi_lag7']

In [ ]:
train.index.nunique()

In [ ]:
lags = ['bui_lag2',  'dmc_lag5', 'dmc_lag7', 'bui_lag4', 'bui_lag5', 'fwi_lag7']
lag_values = {lag: df[lag].values for lag in lags}
for keys, values in lag_values.items():
    if min(values) > 0:
        print('negative lag')

In [ ]:
!pip install pytorch-forecasting

In [ ]:
# Calculate the number of observations per locality
series_lengths = train_filled.groupby('locality')['time_idx'].max() - train_filled.groupby('locality')['time_idx'].min() + 1
print(series_lengths.describe())

In [ ]:
# Check for gaps in time series
def check_time_gaps(group):
    return group['time_idx'].diff().max()

gaps = train_filled.groupby('locality').apply(check_time_gaps)
print(gaps.describe())

In [ ]:
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

# 1. Analyze the original distribution
plt.figure(figsize=(12, 4))
plt.subplot(131)
sns.histplot(train_filled['cfb'], kde=True)
plt.title('Original CFB Distribution')
plt.xlabel('CFB')

# 2. Log transformation (adding a small constant to handle zeros)
log_cfb = np.log1p(train_filled['cfb'])
plt.subplot(132)
sns.histplot(log_cfb, kde=True)
plt.title('Log-transformed CFB')
plt.xlabel('Log(CFB + 1)')

# 4. Quantile transformation
qt = QuantileTransformer(output_distribution='normal')
quantile_cfb = qt.fit_transform(train_filled[['cfb']])

plt.figure(figsize=(12, 4))
plt.subplot(131)
sns.histplot(quantile_cfb, kde=True)
plt.title('Quantile-transformed CFB')
plt.xlabel('Quantile(CFB)')

# 5. Calculate skewness and kurtosis for each transformation
print("Skewness:")
print(f"Original: {stats.skew(train_filled['cfb'])}")
print(f"Log-transformed: {stats.skew(log_cfb)}")
#print(f"Box-Cox: {stats.skew(boxcox_cfb)}")
print(f"Quantile: {stats.skew(quantile_cfb)}")

print("\nKurtosis:")
print(f"Original: {stats.kurtosis(train_filled['cfb'])}")
print(f"Log-transformed: {stats.kurtosis(log_cfb)}")
#print(f"Box-Cox: {stats.kurtosis(boxcox_cfb)}")
print(f"Quantile: {stats.kurtosis(quantile_cfb)}")

# 6. Extreme value analysis
print("\nExtreme Values:")
print(train_filled['cfb'].describe(percentiles=[0.01, 0.05, 0.95, 0.99]))

In [ ]:
# from sklearn.preprocessing import StandardScaler
# # Log transformation
# train_filled['log_cfb'] = np.log1p(train_filled['cfb'])

# # Standard scaling
# scaler = StandardScaler()
# train_filled['scaled_log_cfb'] = scaler.fit_transform(train_filled[['log_cfb']])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from pytorch_forecasting.metrics import MAE, RMSE, SMAPE, QuantileLoss
import torch

# 1. Analyze the distribution of CFB values
zero_cfb_ratio = (train_filled['cfb'] == 0).mean()
print(f"Percentage of zero CFB values: {zero_cfb_ratio * 100:.2f}%")

# 2. Create a binary target for fire occurrence
train_filled['fire_occurrence'] = (train_filled['cfb'] > 0).astype(int)

# 3. Compute class weights for balanced learning
class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_filled['fire_occurrence'])
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# 4. Custom loss function to handle imbalance
class WeightedQuantileLoss(QuantileLoss):
    def __init__(self, zero_weight=1.0, non_zero_weight=10.0, **kwargs):
        super().__init__(**kwargs)
        self.zero_weight = zero_weight
        self.non_zero_weight = non_zero_weight

    def loss(self, y_pred, y_actual):
        base_loss = super().loss(y_pred, y_actual)
        weights = torch.where(y_actual == 0, self.zero_weight, self.non_zero_weight)
        
        # Expand weights to match the shape of base_loss
        weights = weights.unsqueeze(-1).expand_as(base_loss)
        
        return (base_loss * weights).mean()

In [ ]:
train_filled[train_filled['fire_occurrence'] >  0]

In [ ]:
## Define the data schema
static_reals=['elev']
time_varying_known_reals=['month', 'day', 'year']
time_varying_unknown_reals=['cfb', 'temp', 'wd',
       'rh', 'pcuring', 'ros', 'hfi', 'tfc0',
       'sfl', 'bui', 'cfl', 'sfc0', 'dmc', 'sfc',    
       'bfc', 'tfc', 'isi', 'fire_occurrence']
max_prediction_length = 7  # predict 7 days ahead
max_encoder_length = 30  # use 30 days of history

# keep the validation set held-out
training_cutoff = train["time_idx"].max() - max_prediction_length

In [ ]:
from sklearn.preprocessing import StandardScaler
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss

target_normalizer = GroupNormalizer(
    groups=['locality'], 
    transformation="softplus"
)
# target_normalizer.fit(train_filled['cfb'], train_filled)

train_dataset = TimeSeriesDataSet(
    train_filled[lambda x: x.time_idx <= training_cutoff],
    time_idx='time_idx',
    target='cfb',
    group_ids=['locality'],
    static_reals=static_reals,
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=time_varying_unknown_reals,
    max_encoder_length=max_encoder_length,
    min_encoder_length=max_encoder_length//2,  
    max_prediction_length=max_prediction_length,
    min_prediction_length=max_prediction_length,
    target_normalizer=target_normalizer,
    add_relative_time_idx=True,
    add_target_scales=True
)

In [ ]:
# create validation set (predict=True)
validation_dataset = TimeSeriesDataSet.from_dataset(
    train_dataset,
    train_filled,
    predict=True,
    stop_randomization=True,
)

In [ ]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint

# Define the model
# Use the weighted loss in the model definition
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=1e-3,
    hidden_size=64,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=32,
   # output_size=7,  # number of quantiles
    loss=WeightedQuantileLoss(zero_weight=1.0, non_zero_weight=10.0),  # Adjust weights as needed
    log_interval=10,
    reduce_on_plateau_patience=10
)

In [ ]:
# define callbacks
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger(save_dir='/kaggle/working/models')  # log results to a tensorboard
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',   # Metric to monitor
    dirpath='/kaggle/working/checkpoints',  # Directory to save the checkpoints
    filename='best-checkpoint',  # Filename for the best checkpoint
    save_top_k=1,   # Save only the best model
    mode='min'  # Mode of the monitored metric ('min' or 'max')
)
# create trainer
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=0.1,
    limit_train_batches=30,  # run validation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    log_every_n_steps=10,
    callbacks=[lr_logger, early_stop_callback, checkpoint_callback],
    logger=logger,
)

In [ ]:
# Function to print batch shapes
def print_batch_shapes(dataloader, name):
    for i, batch in enumerate(dataloader):
        x, y = batch
        print(f"{name} Batch {i+1}:")
        print(f"  Inputs shape: {x['encoder_cont'].shape}")
        print(f"  Decoder shape: {x['decoder_cont'].shape}")
    #    print(f"  Static real shape: {x['static_cont'].shape}")
      #  print(f"  Targets shape: {len(y)}")
        # Print only the first batch to avoid too much output
        break


In [ ]:
# Create dataloaders
batch_size = 64
train_dataloader = train_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=4)

# Print shapes for train and validation dataloaders
print_batch_shapes(train_dataloader, "Train")
print_batch_shapes(val_dataloader, "Validation")

# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
# load the best model w.r.t. the validation loss
best_model_path = checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
predictions = tft.predict(val_dataloader)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictions = predictions.to(device)
actuals = torch.cat([y[0] for _, y in iter(val_dataloader)]).to(device)

# Initialize metrics on the same device
rmse = RMSE().to(device)
mae = MAE().to(device)

# Calculate and print RMSE and MAE
print(f"RMSE: {rmse(predictions, actuals)}")
print(f"MAE: {mae(predictions, actuals)}")

In [ ]:
# Get predictions and ensure they are on the same device
val_prediction_results = best_tft.predict(val_dataloader).to(device)

# Compute the mean absolute error
mae = (actuals - val_prediction_results).abs().mean()
print(f"Validation MAE: {mae.item()}")

In [ ]:
val_prediction_results = best_tft.predict(val_dataloader, mode="raw", return_x=True).to(device)

In [ ]:
# Plot actuals vs prediction and attention
for idx in range(1):
    fig, ax = plt.subplots(figsize=(23,5))
    best_tft.plot_prediction(val_prediction_results.x, # network input
                            val_prediction_results.output, # network output
                            idx=idx,
                            add_loss_to_title=True,
                            ax=ax);

In [ ]:
# plot variable importance
interpretation = best_tft.interpret_output(val_prediction_results.output, reduction="sum")
best_tft.plot_interpretation(interpretation)